In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tuktoyaktuk,CA,2021-11-03 23:58:58,69.4541,-133.0374,17.60,93,20,10.36
1,1,Kirkenaer,NO,2021-11-03 23:58:58,60.4564,12.0582,42.28,95,100,2.59
2,2,Ushuaia,AR,2021-11-03 23:57:18,-54.8000,-68.3000,46.06,66,40,26.46
3,3,Chokurdakh,RU,2021-11-03 23:58:59,70.6333,147.9167,-9.92,98,95,7.56
4,4,Grand Gaube,MU,2021-11-03 23:58:59,-20.0064,57.6608,73.80,76,3,3.00


In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
positive_temp_df=city_data_df.loc[city_data_df["Max Temp"]>=0]

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = positive_temp_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = positive_temp_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [13]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [15]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Grand Gaube,MU,2021-11-03 23:58:59,-20.0064,57.6608,73.80,76,3,3.00
5,5,Jalu,LY,2021-11-03 23:58:59,29.0331,21.5482,75.07,26,0,8.39
8,8,Bosaso,SO,2021-11-03 23:59:01,11.2842,49.1816,76.33,46,35,4.18
13,13,East London,ZA,2021-11-03 23:56:34,-33.0153,27.9116,62.64,86,3,6.22
14,14,Vila Franca Do Campo,PT,2021-11-03 23:59:03,37.7167,-25.4333,68.20,88,100,11.36
16,16,Zhicheng,CN,2021-11-03 23:59:03,30.2956,111.5047,59.59,87,97,2.98
17,17,Mataura,NZ,2021-11-03 23:59:04,-46.1927,168.8643,64.47,34,33,1.72
18,18,Atuona,PF,2021-11-03 23:59:04,-9.8000,-139.0333,78.31,70,39,15.17
19,19,Hasaki,JP,2021-11-03 23:59:04,35.7333,140.8333,57.60,51,57,6.67
22,22,Georgetown,MY,2021-11-03 23:56:46,5.4112,100.3354,77.90,92,20,0.00


In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Grand Gaube,MU,73.80,-20.0064,57.6608,
5,Jalu,LY,75.07,29.0331,21.5482,
8,Bosaso,SO,76.33,11.2842,49.1816,
13,East London,ZA,62.64,-33.0153,27.9116,
14,Vila Franca Do Campo,PT,68.20,37.7167,-25.4333,
16,Zhicheng,CN,59.59,30.2956,111.5047,
17,Mataura,NZ,64.47,-46.1927,168.8643,
18,Atuona,PF,78.31,-9.8000,-139.0333,
19,Hasaki,JP,57.60,35.7333,140.8333,
22,Georgetown,MY,77.90,5.4112,100.3354,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECbw4x2ZcH5D1xYhBiJbK0Ahv9553Niwno1t42XVC0kNEty0HxHNhvtITNEYqKvddrqHqnCJvaXflvfLCZ_4WyPxEA0uy_5BrmC5X1olYU5ZdoHPj5muwscG5myv9Dja4cDLBVecPC_KZSBSInrBy-O8FS4quwby5z8RnWnrLvfXW0cb6Jv46t4AnziW-OZhtNDAQNNeegd42RlHRE9Cv4QhwLXbXm1XrVfMW9inyFumWL7Dsqe4979FK1R0UG27gxX7n7fR3YZdN1I3y2zKvcceZnLRUGv96Prf_sAqzDhtCJr6IuPlVFCK8zi52aByW9D_DS0Ah00MhG0z1D4q347-XC7rT3va4H_94H3sXRipvKFIsbzYW9c9-vcIlvQHdI-cEVKBhiOEEs8okGVSlQ5COgbX33t98Spm2la-oVCtJHiIxmUlne8',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [21]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [22]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECsKI0cAKf7ZP7oRD175ayDK1IYtaf5iv_GIqiNPPqrYeryPiBQ-GMIsRuBMU9SeEg5Yb2Q55z8wXaptS1bS2Iy5ppfv7Uo9GJHWd4M-h8LRUqsxEcHJ5uEw_J6pZcn1ui1gVmsidUcJC26EWRRmvjowme_lxJ9dmVQlu4qhDdenI_MHAdZegJNS8_zVk8oUG6Bp3PKY-LVjL7lo_CGLqb1-YtHjnDwAeSbOw0boYtQBehEwqojRKR3vUFw6xIndxfqxghqW20ZQebaLzEaSvSfYJEr78Wec2i-DRN6NLBMuoQKWeZChsuDYUA-Hn4NnlW_kqJxrlFGcANL5VxrT8kZputWy3zEiMs6QABXQtUb2o0ch-WnZEaBIJnMOyCQrGmtQsmK6L6iQvoRa5kzNywy6fMmD4EcgBexd3Q09o5vrzOlucWliBra',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [26]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [30]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))